# Analyzing the dataset of LoL's champions

https://www.kaggle.com/datasets/uskeche/leauge-of-legends-champions-dataset?resource=download

In [1]:
import pandas as pd

Read the data

In [2]:
Dat = pd.read_csv("LoLChampions.csv")
Dat = Dat.drop("Id", axis=1)

---

## Data quality and data preprocessing

The first thing that we should do is to check the quality of the data set.

In [3]:
Dat.head()

,Name,Class,Style,Difficulty,DamageType,Damage,Sturdiness,Crowd-Control,Mobility,Functionality
0,Aatrox,Warrior,3,2,P,3,3,2,2,2.0
1,Ahri,Mage,10,2,M,3,1,2,3,1.0
2,Akali,Assasin,7,2,PM,3,1,1,3,1.0
3,Alistar,Tank-Support,7,1,M,1,3,3,1,2.0
4,Amumu,Tank,9,1,M,2,3,3,1,1.0


In [4]:
Dat.describe()

,Style,Difficulty,Damage,Sturdiness,Crowd-Control,Mobility,Functionality
count,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,144.000000
mean,6.544828,1.882759,2.434483,1.662069,2.013793,1.717241,1.465278
std,2.806192,0.702213,0.715054,0.774782,0.726351,0.742588,0.668086
min,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,4.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000
50%,7.000000,2.000000,3.000000,1.000000,2.000000,2.000000,1.000000
75%,9.000000,2.000000,3.000000,2.000000,3.000000,2.000000,2.000000
max,10.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000


We can observe that there are missing values in the Functionality variable.

In [5]:
Dat[Dat["Functionality"].isnull()]

,Name,Class,Style,Difficulty,DamageType,Damage,Sturdiness,Crowd-Control,Mobility,Functionality
23,Ekko,Mage-Warrior,8,3,M,3,2,2,3,NaN


There are other issues regarding the quality of the data. For example, the type of the variables, several of them should be categorical.

For now, we are going to keep the data set as it is, since these issues won't compromise our analysis.

In [6]:
Dat.dtypes

Name              object
Class             object
Style              int64
Difficulty         int64
DamageType        object
Damage             int64
Sturdiness         int64
Crowd-Control      int64
Mobility           int64
Functionality    float64
dtype: object

Imagine that we select a champion at random (all of them with the same probability of being selected).

---

## Classical probability

By classical probability, if we want to estimate the probability of the event $A$, we can do it as: $$P(A)=\frac{1}{n}\sum_{i=1}^n 1(\text{observation}_i\text{ satisfies A}).$$

Which is just the average of $1(\text{observation}\text{ satisfies A})$.

What is the prob. of select a class warrior?

In [7]:
round((Dat["Class"]=="Warrior").mean(), 4)

0.2138

---

## Conditional Probability

For conditional probabilities, we have, at least, two different ways to calculate them.

For instance, if we want to calculate the probability of $P(A|B)$, we can do it as:

1. Filter for the observations where the event $B$ happened. Then, calculate the probability of $A$ considering only those observations.
2. Calculate it using the definition of conditional probability, i.e. $$P(A|B)=\frac{P(A,B)}{P(B)}$$

What is the prob. of selecting a class warrior given that it is of difficulty 1?

Let's do it first using the filtering approach:

In [8]:
# Filter the observtions to consider only the champions where difficulty=1
DatDifficult1 = Dat[Dat["Difficulty"]==1]

# Calculate the probability of selecting a class warrior, considering only the observations previosly filtered
round((DatDifficult1["Class"]=="Warrior").mean(), 4)

0.2444

Using the definition:

1. We need to calculate the join probability of selecting a class warrior and a difficulty equal 1: $$P(\text{warrior and difficulty=1})$$
2. We need to calculate the marginal probability of selecting a champion of difficulty 1 $$P(\text{difficulty=1})$$
3. We need to divide the former probability by the later: $$P(\text{warrior|difficulty=1})=\frac{P(\text{warrior and difficulty=1})}{P(\text{difficulty=1})}$$

In [9]:
PWarriorAndDifficult1 = ((Dat["Class"]=="Warrior") & (Dat["Difficulty"]==1)).mean()
PDifficult1 = (Dat["Difficulty"]==1).mean()

PWarriorGivenDifficult1 = PWarriorAndDifficult1 / PDifficult1

round(PWarriorGivenDifficult1, 4)

0.2444

---

## Multiplication law or chain rule

Consider that we want to calculate the probability of the events $A$, $B$, and $C$ at the same time, i.e. $P(A\cap B\cap C)$.

The multiplication law, says that such probability can be calculated as: $$P(A\cap B\cap C)=P(A|B,C)P(B|C)P(C)$$

Note that, it doesn't matter the order in which we condition.

What is the prob. of selecting a class warrior, difficulty 1, and style 4?

First, let's calculate it using the deffinition of classical probability

In [10]:
round(((Dat["Class"]=="Warrior") & (Dat["Difficulty"]==1) & (Dat["Style"]==4)).mean(), 4)

0.0207

Now, let's calculate it using the multiplication law:

In [11]:
# Probability of being warrior given that it is difficulty 1 and style 4
print(((Dat["Class"]=="Warrior") & (Dat["Difficulty"]==1) & (Dat["Style"]==4)).mean() / ((Dat["Difficulty"]==1) & (Dat["Style"]==4)).mean())

# Probability of difficulty 1 given style 4
print(((Dat["Difficulty"]==1) & (Dat["Style"]==4)).mean() / (Dat["Style"]==4).mean())

# Probability of style 4
print((Dat["Style"]==4).mean())

0.5
0.46153846153846156
0.0896551724137931


Thus, the probability that we are looking for should be the product of these three probabilities

In [12]:
round(0.5 * 0.46153846153846156 * 0.0896551724137931, 4)

0.0207

---

## Law of total probability

Law of total probability stays that, if $B_1,B_2,B_3,\ldots$ form a partition of $\Omega$, i.e. $\cup_{i}B_i=\Omega$ and $B_i\cap B_j=\emptyset$ for all $i\neq j$. Then, $$P(A)=\sum_iP(A|B_i)P(B_i)=\sum_i P(A,B_i)$$

What is the prob. of selecting a warrior?

First, let's calculate this probability using classical probability

In [13]:
round((Dat["Class"]=="Warrior").mean(), 4)

0.2138

Now, let's calculate it using law of total probability.

Note that, every champion should be of difficulty 1, 2, or 3. So the events $B_i:$ the champion is of difficulty $i$, form a partition of $\Omega$.

Thus, the probability of selecting a warrior can be calculate it as: $$P(\text{warrior})=P(\text{warrior and difficulty 1}) + P(\text{warrior and difficulty 2}) + P(\text{warrior and difficulty 3})$$

In [14]:
print(((Dat["Class"]=="Warrior") & (Dat["Difficulty"]==1)).mean())
print(((Dat["Class"]=="Warrior") & (Dat["Difficulty"]==2)).mean())
print(((Dat["Class"]=="Warrior") & (Dat["Difficulty"]==3)).mean())

0.07586206896551724
0.09655172413793103
0.041379310344827586


In [15]:
round(0.07586206896551724 + 0.09655172413793103 + 0.041379310344827586, 4)

0.2138

---

## Bayes' theorem

What is the probability of selecting a warrior given that it is of difficulty 1?

We already calculated that probability previously.

In [16]:
PWarriorGivenDifficult1

0.24444444444444444

We are going to calculate it again. But this time, we are going to use Bayes' theorem.

Accordign to Bayes' theorem, the probability that we are looking for, can be calculated as:

$$P(\text{Warrior|Difficulty=1})=\frac{P(\text{Difficulty=1|Warrior})P(\text{Warrior})}{P(\text{Difficulty=1})}$$

In [17]:
PDifficult1GivenWarrior = ((Dat["Class"]=="Warrior") & (Dat["Difficulty"]==1)).mean() / (Dat["Class"]=="Warrior").mean()
PWarrior = (Dat["Class"]=="Warrior").mean()
PDifficult1 = (Dat["Difficulty"]==1).mean()

round(PDifficult1GivenWarrior * PWarrior / PDifficult1, 4)

0.2444